In [ ]:
!pip install fastapi uvicorn nest-asyncio pyngrok transformers textblob
!python -m textblob.download_corpora

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.1 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Unzipping corpora/conll2000.zip.
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Unzipping corpora/movie_reviews.zip.
Finished.


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from fastapi.middleware.cors import CORSMiddleware
from transformers import pipeline
from textblob import TextBlob

import nest_asyncio
import uvicorn
from pyngrok import ngrok

# Initialize FastAPI
app = FastAPI()
nest_asyncio.apply()

# CORS for frontend access
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_methods=["*"],
    allow_headers=["*"],
)

# Load AI model
chatbot = pipeline("text-generation", model="microsoft/DialoGPT-medium")

class Message(BaseModel):
    text: str

def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.3:
        return "positive"
    elif polarity < -0.3:
        return "negative"
    return "neutral"

@app.post("/chat")
def chat(msg: Message):
    sentiment = get_sentiment(msg.text)
    base = chatbot(msg.text, max_length=50, truncation=True)[0]["generated_text"]

    if sentiment == "positive":
        prefix = "That's great to hear! "
    elif sentiment == "negative":
        prefix = "I'm sorry you're feeling that way. "
    else:
        prefix = "I understand. "

    return {"response": prefix + base.strip()}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/863M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/863M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
public_url = "https://abc123.ngrok.io"
print("Backend running at:", public_url)

Backend running at: https://abc123.ngrok.io


In [ ]:
from IPython.display import HTML

public_url = "https://abc123.ngrok.io"  # <-- Replace with your ngrok URL

HTML(f"""
<!DOCTYPE html>
<html>
<head>
  <style>
    body {{ font-family: Arial; padding: 10px; }}
    #chatBox {{ border: 1px solid #ccc; padding: 10px; height: 300px; overflow-y: scroll; background: #f9f9f9; }}
    input, button {{ padding: 8px; margin-top: 5px; }}
  </style>
</head>
<body>
  <h3>AI Emotionally Adaptive Companion</h3>
  <div id="chatBox"></div>
  <input id="userInput" placeholder="Say something..." />
  <button onclick="sendMessage()">Send</button>

  <script>
    const chatBox = document.getElementById("chatBox");

    async function sendMessage() {{
      const input = document.getElementById("userInput");
      const message = input.value;
      input.value = "";

      chatBox.innerHTML += "<p><strong>You:</strong> " + message + "</p>";

      const response = await fetch("{public_url}/chat", {{
        method: "POST",
        headers: {{ "Content-Type": "application/json" }},
        body: JSON.stringify({{ text: message }})
      }});
      const data = await response.json();
      chatBox.innerHTML += "<p><strong>Companion:</strong> " + data.response + "</p>";
      chatBox.scrollTop = chatBox.scrollHeight;
    }}
  </script>
</body>
</html>
""")